# Librerias

In [1]:
import pandas as pd
import numpy as np
#import nltk
#import spacy
import re
import unicodedata
#nlp = spacy.load('pt_core_news_sm')
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

# data

In [3]:
df=pd.read_csv('data/train.csv')

# Filter

In [4]:
temp1=df[df['label_quality']=='unreliable']['category'].tolist()
temp2=df[df['label_quality']=='reliable']['category'].tolist()

In [5]:
cat_not_in_reliable=list(set(temp1) - set(temp2))

In [6]:
cat_count=df[(df['label_quality']=='reliable')|(df['category'].isin(cat_not_in_reliable))]['category'].value_counts()

In [7]:
cat_low_count=cat_count[cat_count<4000].index.tolist()

In [8]:
df=df[(df['label_quality']=='reliable')|(df['category'].isin(cat_not_in_reliable))|(df['category'].isin(cat_low_count))]

In [9]:
#df_portuguese = df[df['language']=='portuguese']
#df_portuguese=df_portuguese.groupby('category').filter(lambda x:len(x)>=20)
#counts_portuguese = df_portuguese['category'].value_counts()

all_cat=df['category'].value_counts()

all_cat=all_cat[all_cat<=400].index.tolist()

df_portuguese_temp        = df[df['language']=='portuguese']
counts_portuguese = df_portuguese_temp['category'].value_counts()
low_portuguese=counts_portuguese[counts_portuguese<200]

df_portuguese=df[(df['language']=='portuguese')|\
              (~df['category'].isin(counts_portuguese.index))|\
              (df['category'].isin(all_cat))|\
              (df['category'].isin(low_portuguese.index))]

# ML

## Preprocessing

In [10]:
def clean_text(text) :
    text=unicodedata.normalize('NFKD', str(text)).encode('ascii', errors='ignore').decode('utf-8')\
    .lower().replace(r'\\n','').replace(r'-',' ').replace(r'.','').strip()
    text = re.sub("[^a-zA-Z]", " ", str(text))
    return " ".join(text.split())

In [11]:
def lemmatization(text):
    txt = nlp(text)
    lemmatize_sentence=[]
    for token in txt:
        lemmatize_sentence.append(token.lemma_)
    return " ".join(lemmatize_sentence)

## Sample

In [12]:
sampling_portuguese={}
counts_portuguese  = df_portuguese['category'].value_counts()
for cat in counts_portuguese.index:
    if counts_portuguese.loc[cat]>2200:
        sampling_portuguese[cat]=2200
    else:
        sampling_portuguese[cat]=counts_portuguese.loc[cat]

In [13]:
df_portuguese['category'].value_counts()

CAR_AV_RECEIVERS                        17788
WHEELS_BEARINGS                         17499
WALL_CLOCKS                             17335
FISHING_LINES                           16703
CV_JOINTS                               16553
                                        ...  
CONSTRUCTION_LIME_BAGS                    206
COLD_FOOD_AND_DRINK_VENDING_MACHINES      162
PAINTBALL_SMOKE_GRENADES                  154
COMMERCIAL_POPCORN_MACHINES               141
HAMBURGER_FORMERS                         109
Name: category, Length: 1588, dtype: int64

In [14]:
from imblearn.pipeline import Pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE,BorderlineSMOTE
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

In [15]:
from sklearn.metrics import balanced_accuracy_score

In [16]:
X_portuguese = df_portuguese['title']
y_portuguese = df_portuguese['category']

In [17]:
rus_portuguese = RandomUnderSampler(sampling_strategy=sampling_portuguese,random_state=42)
X_res_portuguese, y_res_portuguese = rus_portuguese.fit_resample(X_portuguese.values.reshape(-1, 1), y_portuguese)

In [18]:
#PORTUGUESE
X_res_portuguese=pd.DataFrame(X_res_portuguese)
X_res_portuguese.columns=['title']
X_res_portuguese['title']=X_res_portuguese['title'].astype('str')
y_res_portuguese=pd.Series(y_res_portuguese)

In [19]:
print(y_res_portuguese.value_counts())

YOGA_MATS                               2200
GRILL_GRATES                            2200
SHOWER_CURTAINS                         2200
SHAVING_CREAMS                          2200
SPARK_PLUGS                             2200
                                        ... 
CONSTRUCTION_LIME_BAGS                   206
COLD_FOOD_AND_DRINK_VENDING_MACHINES     162
PAINTBALL_SMOKE_GRENADES                 154
COMMERCIAL_POPCORN_MACHINES              141
HAMBURGER_FORMERS                        109
Length: 1588, dtype: int64


## lemmatization

In [20]:
#spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS

In [21]:
#X_res_portuguese['title']=X_res_portuguese['title'].apply(lemmatization)

## CLEAN

In [22]:
X_res_portuguese['title']=X_res_portuguese['title'].apply(clean_text)

## train_test_split

In [23]:
X_train_portuguese, X_test_portuguese, y_train_portuguese, y_test_portuguese = train_test_split(X_res_portuguese['title'],
                                                                                                y_res_portuguese,
                                                                                                test_size=0.46,
                                                                                                random_state=42,
                                                                                                stratify=y_res_portuguese)

In [24]:
print('X_train_portuguese : ',X_train_portuguese.shape)

X_train_portuguese :  (1676952,)


In [25]:
print('y_train_portuguese : ',y_train_portuguese.shape)

y_train_portuguese :  (1676952,)


In [26]:
print('y_train_portuguese : \n ',y_train_portuguese.value_counts())

y_train_portuguese : 
  YOGA_MATS                               1188
MICROWAVES                              1188
CLASSIC_CARS                            1188
RAM_MEMORY_MODULES                      1188
STARTERS                                1188
                                        ... 
CONSTRUCTION_LIME_BAGS                   111
COLD_FOOD_AND_DRINK_VENDING_MACHINES      87
PAINTBALL_SMOKE_GRENADES                  83
COMMERCIAL_POPCORN_MACHINES               76
HAMBURGER_FORMERS                         59
Length: 1588, dtype: int64


In [27]:
print('y_test_portuguese : ',y_test_portuguese.nunique())

y_test_portuguese :  1588


In [28]:
print('y_test_portuguese : \n',y_test_portuguese.value_counts())

y_test_portuguese : 
 YOGA_MATS                               1012
PORTABLE_HANGING_SCALES                 1012
RAM_MEMORY_MODULES                      1012
STARTERS                                1012
VACUUM_TUBES                            1012
                                        ... 
CONSTRUCTION_LIME_BAGS                    95
COLD_FOOD_AND_DRINK_VENDING_MACHINES      75
PAINTBALL_SMOKE_GRENADES                  71
COMMERCIAL_POPCORN_MACHINES               65
HAMBURGER_FORMERS                         50
Length: 1588, dtype: int64


## MultinomialNB

In [25]:
bayes_text_clf =  make_pipeline_imb([('CountVectorizer', CountVectorizer(encoding='latin-1',
                                                               max_df=0.90,
                                                               min_df=5,
                                                               max_features=100000,
                                                               #stop_words=list(spacy_stopwords),
                                                              )),
                                     ('SMOTE',SMOTE(random_state=42)),
                                     ('MultinomialNB',  MultinomialNB())])

In [27]:
bayes_portuguese_clf=bayes_text_clf.fit(X_train_portuguese,y_train_portuguese)

In [ ]:
y_pred_portuguese = bayes_portuguese_clf.predict(X_test_portuguese.head(X_test_portuguese.shape[0]//2))
balanced_accuracy_score(y_test_portuguese.head(X_test_portuguese.shape[0]//2), y_pred_portuguese)

In [ ]:
y_pred_portuguese = bayes_portuguese_clf.predict(X_test_portuguese.tail(X_test_portuguese.shape[0]//2))
balanced_accuracy_score(y_test_portuguese.tail(X_test_portuguese.shape[0]//2), y_pred_portuguese)

In [ ]:
y_pred_portuguese = bayes_portuguese_clf.predict(X_test_portuguese)
balanced_accuracy_score(y_test_portuguese, y_pred_portuguese)

In [30]:
LinearSVC_portuguese_clf=LinearSVC_text_clf.fit(X_train_portuguese,y_train_portuguese)

In [31]:
y_test_portuguese.nunique()

1588

In [32]:
X_test_portuguese.shape

(1399530,)

## SVM Linear TfidfVectorizer SMOTE

In [29]:
LinearSVC_text_clf =  make_pipeline_imb([('TfidfVectorizer', TfidfVectorizer(encoding='latin-1',
                                                               max_df=0.90,
                                                               min_df=5,
                                                               max_features=150000,
                                                               #stop_words=list(spacy_stopwords),
                                                              )),
                                     ('SMOTE',SMOTE(random_state=42)),
                                     ('LinearSVC',  LinearSVC())])

In [30]:
LinearSVC_portuguese_clf=LinearSVC_text_clf.fit(X_train_portuguese,y_train_portuguese)

In [31]:
y_test_portuguese.nunique()

1588

In [32]:
X_test_portuguese.shape

(1428516,)

In [33]:
y_pred_portuguese = LinearSVC_portuguese_clf.predict(X_test_portuguese.head(X_test_portuguese.shape[0]//2))
balanced_accuracy_score(y_test_portuguese.head(X_test_portuguese.shape[0]//2), y_pred_portuguese)

0.8555814922105924

In [34]:
0.8519735090261912

0.8519735090261912

In [35]:
y_pred_portuguese = LinearSVC_portuguese_clf.predict(X_test_portuguese.tail(X_test_portuguese.shape[0]//2))
balanced_accuracy_score(y_test_portuguese.tail(X_test_portuguese.shape[0]//2), y_pred_portuguese)

0.8553832317964092

In [36]:
0.8516783227482038

0.8516783227482038

# Save Model

In [37]:
import pickle

In [38]:
filename = 'LinearSVC_tfidf_SMOTE_MORE_FEATURES_DATAportuguese_clf_2.sav'
pickle.dump(LinearSVC_portuguese_clf, open(filename, 'wb'))